In [18]:
from langchain.chat_models import ChatOpenAI, ChatOllama
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model="mistral",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [22]:
examples = [
    {
        "question": "당신은 프랑스에 대해서 무엇을 알고있나요?",
        "answer": """
		내가 알고있는것:
		수도: 파리
		언어: 불어
		음식: 와인과 치즈
		통화: 유로
		""",
    },
    {
        "question": "당신은 이탈리아에 대해서 무엇을 알고있나요?",
        "answer": """
		내가 알고있는 것:
		수도: 로마
		언어: 이탈리아어
		음식: 피자와 파스타
		통화: 유로
		""",
    },
    {
        "question": "당신은 그리스에 대해서 무엇을 알고있나요?",
        "answer": """
		내가 알고있는 것은:
		수도: 아테네
		언어: 그리스어
		음식: 수플라키와 페타치즈
		통화: 유로
		""",
    },
]


class RandomExamleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_templete = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_templete)

example_selector = RandomExamleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="당신은 {country}에 대해서 무엇을 알고있나요?",
    input_variables=["country"],
)

prompt.format(country="독일")

'\n    Human: 당신은 프랑스에 대해서 무엇을 알고있나요?\n    AI: \n\t\t내가 알고있는것:\n\t\t수도: 파리\n\t\t언어: 불어\n\t\t음식: 와인과 치즈\n\t\t통화: 유로\n\t\t\n\n\n당신은 독일에 대해서 무엇을 알고있나요?'